In [1]:
import pandas as pd
import numpy as np
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score
import os

In [2]:
def to_adjascency_mat(edges, numOfNodes):
    
    m = np.zeros((numOfNodes, numOfNodes))
    
    for e in edges:
        i, j = e[0], e[1]
        m[i-1,j-1] = 1
    return m

# Crate target filename and Load data
#dataFile = '../Data/Sachs/1. cd3cd28.xls'
origFolder = ''
genProc = '4'
dataFile = os.path.join('.', 'Data', origFolder, genProc, 'data.npy')
fileType = dataFile.split('.')[-1]
# Use appropriate laoding procedure according to file type
if fileType == 'xls':
    df = pd.read_excel(dataFile)
elif fileType == 'npy' or fileType == 'txt':
    data = np.load(dataFile)
    df = pd.DataFrame(data).astype(np.float32)

# Construct columns and normalize them for convergence speed
df.columns = list('ABCDEFGHIJK') #[i for i in range(0, df.shape[1])] # REname columns to numbers for easy of use and readability
normalized_df=((df-df.min())/(df.max()-df.min())).astype(np.float32)
#print(normalized_df, df.max())
#data = normalized_df.to_numpy()
print(normalized_df)
#print(data)

            A         B         C         D         E         F         G  \
0    0.035406  0.039974  0.053771  0.069875  0.028436  0.077679  0.062940   
1    0.024539  0.027288  0.025889  0.032732  0.004984  0.437946  0.554930   
2    0.091265  0.098610  0.037009  0.045334  0.024951  0.102232  0.095784   
3    0.032146  0.033490  0.057800  0.071755  0.017524  0.307143  0.333735   
4    0.029972  0.034759  0.014462  0.017698  0.043322  0.205357  0.257993   
..        ...       ...       ...       ...       ...       ...       ...   
718  0.081701  0.086770  0.208655  0.209936  0.105537  0.295982  0.343120   
719  0.078659  0.075635  0.044746  0.044450  0.013648  0.220982  0.192305   
720  0.045187  0.044766  0.037976  0.048761  0.022997  0.037455  0.011395   
721  0.254928  0.260705  0.024922  0.030079  0.009642  0.407143  0.371942   
722  0.297529  0.263102  0.172715  0.217674  0.268730  0.199554  0.203700   

            H         I         J         K  
0    0.032829  0.057205  0.06

In [3]:
# create data sample with 9 random variables:
d = 5
randArr = np.random.uniform(low=0, high=1, size=(500, d))
#randArr = np.random.randint(0, 5, size=(1000, d))
data = pd.DataFrame(randArr, columns=list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')[:d])

# add 10th dependent variable
data['C'] = data['A'] * data['B']

#print(df)
est = HillClimbSearch(data, scoring_method=BicScore(data))
#best_model = est.estimate(max_iter=10)
best_model = None
best_model = est.estimate(max_indegree=2, max_iter=10)
#print(sorted(best_model.nodes()))
#print(best_model.edges())
#j = list(best_model.edges())
# search a model with restriction on the number of parents:
#est.estimate(max_indegree=1).edges()


 40%|████      | 4/10 [02:06<03:09, 31.65s/it]


In [4]:
print(best_model.edges())

[('B', 'C'), ('B', 'A'), ('D', 'E'), ('E', 'B')]


In [4]:
est = HillClimbSearch(normalized_df, scoring_method=BicScore(normalized_df))
best_model = est.estimate(max_indegree=2,max_iter=10)

100%|██████████| 10/10 [02:36<00:00, 15.69s/it]


In [ ]:
def alpha_to_num(L):
    letters = list('ABCDEFGHIJKLMNOP')
    for i in range(len(letters)):
        if L == letters[i]:
            return i
    return -1
print(best_model.edges())

l = best_model.edges()

l2 = [ (alpha_to_num(x),alpha_to_num(y)) for x,y in l ]
print(l2)

In [9]:


m = to_adjascency_mat(l2, 11)
#saveFileBase = './Results/mmhc_sachs_data_1'
saveFileBase = './Results'
saveFile = os.path.join(saveFileBase, '_'.join(('mmhc', genProc, 'predG')))
np.save(saveFileBase+'.npy', m)
np.savetxt(saveFileBase, m)
print(m)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [ ]:
from rpy2.robjects import default_converter
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# use the default conversion rules to which the pandas conversion
# is added
with localconverter(default_converter + pandas2ri.converter) as cv:
    dataf = robjects.r["data"]